## User Input: Number of Clusters

In [1]:
# number of clusters
num_clusters = 5

In [4]:
# import packages
import pandas as pd
import utils
import pickle
import yake
from sklearn.feature_extraction.text import CountVectorizer

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
# Load grouped data by component from saved pickle

with open("ClustersTfidf.pickle", "rb") as pickle_file:
    grouped_data = pickle.load(pickle_file)

# #Uncomment to inspect Doc2Vec    
# with open("ClustersDoc2Vec.pickle", "rb") as pickle_file:
#     grouped_data = pickle.load(pickle_file)

In [ ]:
# Examine by Subgroup
groups = grouped_data.keys()

# Visualize: What's in the clusters? 

In [ ]:
# Show first 5 requirements in each cluster
for g in groups:
    print(g)
    for count in range(num_clusters):
        print(utils.inspect_cluster(grouped_data, g, count)[:5])

In [ ]:
### VISUALIZE ### What requirements are in the cluster?

# Update the following to inspect the clusters
cluster_number = 3 # change this to view different clusters
view = 'Requirement' # Uncomment to view prepocessed message 
# view = 'Summary' # Uncomment to view original summary message 

utils.inspect_cluster(grouped_data, g, cluster_number)

# list(grouped_data[g][view].loc[grouped_data[g]['predicted']==cluster_number])

# Cluster Descriptions
## Keyword Extraction Using YAKE
You can get keywords to represent your clusters by using keyword extraction. YAKE is a good one, and it’s easy to use. First, you’ll need to convert your vector back to its original text. If you didn’t lemmatize during your preprocessing, it’s a good idea to do that before applying keyword extraction. Then, combine all the text in your cluster into a single text blob or “document,” meaning, you’ll need to collapse your list of lists into a single element. Then, simply use YAKE or another keyword extractor to get the most representative words for your cluster. If you don’t see anything meaningful here, your clustering algorithm may be picking up on patterns you don’t care about. Add any irrelevant words to your list of words to remove and try again. 


In [ ]:
# Create clusters_data to more easily access the requirements in each cluster
clusters_data = []
max_index = 4
for cluster_number in range(max_index):
    clusters_data.append(list(grouped_data[g]['Requirement'].loc[grouped_data[g]['predicted']==cluster_number]))

In [ ]:
clusters_data[0]

In [ ]:
### Attempting to use Erin's code to print out keywords with POS - needs debugging
# ### Using Yake to to vectorize and save vocab for each group
# yake_vocab = {}
# language = "en"
# max_ngram_size = 1
# deduplication_thresold = 0.9
# deduplication_algo = 'seqm'
# windowSize = 1
# numOfKeywords = 5 # changed to 5 so it doesn't return every word in the Requirement

# ### pull this out of the for loop for efficiency
# custom_kw_extractor = yake.KeywordExtractor(lan=language, 
#                                         n=max_ngram_size, 
#                                         dedupLim=deduplication_thresold, 
#                                         dedupFunc=deduplication_algo, 
#                                         windowsSize=windowSize, 
#                                         top=numOfKeywords, 
#                                         features=None)

# vocab = {}
# for g in groups:
#     vocab[g] = []
#     keywords = []
#     tags = []
#     for i, text in enumerate(grouped_data[g]['Requirement']):
#         kw = custom_kw_extractor.extract_keywords(text)
# #         keywords.append(' '.join([word for word, score in kw]))
#         keywords.append([word for word, score in kw])
#         text = nltk.word_tokenize(text)
        
#         tmp_tags = []
#         for keyword in [word for word, score in kw]:
#             try:
#                 tmp_tags.append(grouped_data[g].iloc[i]['tags'][text.index(keyword)])
#             except:
#                 continue
#         tags.append(tmp_tags)  
#     for i in range(len(keywords)):
#         new_requirement = []
#         for j in range(len(keywords[i])):
#             try:
#                 new_requirement.append((keywords[i][j], tags[i][j]))
#             except:
#                 print(g, i, j)
#                 print(keywords[i], tags[i])
#         vocab[g].append(new_requirement)
    

In [ ]:
### Using Yake to on clusters and save keywords for each cluster


language = "en"
max_ngram_size = 1
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 4
yake_keywords_clusters = {}

custom_kw_extractor = yake.KeywordExtractor(lan=language, 
                                            n=max_ngram_size, 
                                            dedupLim=deduplication_thresold, 
                                            dedupFunc=deduplication_algo, 
                                            windowsSize=windowSize, 
                                            top=numOfKeywords, 
                                            features=None)
for cluster_number in range(max_index):
    keywords = []
    for text in clusters_data[cluster_number]:
        kw = custom_kw_extractor.extract_keywords(text)
        keywords.append(' '.join([word for word, score in kw]))
    yake_keywords_clusters[cluster_number] = kw


    vectorizer = CountVectorizer(stop_words='english', ngram_range=(0,1))
    X = vectorizer.fit_transform(keywords)
    ## X will be array of word embeddings
#     print('Word embeddings shape: ', end=' ')
#     print(cluster_number, X.shape)
    document_term_matrix = pd.DataFrame(X.toarray(),
                                        columns=vectorizer.get_feature_names())
    
# with open("yake_vocabulary.txt", "w") as text_file:
#     text_file.write(str(yake_vocab))

In [ ]:
yake_keywords_clusters

In [ ]:
### Using Yake to to vectorize and save vocab for each group
# yake_vocab={}

# custom_kw_extractor = yake.KeywordExtractor(lan=language, 
#                                             n=max_ngram_size, 
#                                             dedupLim=deduplication_thresold, 
#                                             dedupFunc=deduplication_algo, 
#                                             windowsSize=windowSize, 
#                                             top=numOfKeywords, 
#                                             features=None)
# for g in groups:
#     
#     keywords = []
#     for text in grouped_data[g]['preprocessed']:
#         kw = custom_kw_extractor.extract_keywords(text)
#         keywords.append(' '.join([word for word, score in kw]))

#     vectorizer = CountVectorizer(stop_words='english', ngram_range=(0,1))
#     X = vectorizer.fit_transform(keywords)
#     ## X will be array of word embeddings
# #     print('Word embeddings shape: ', end=' ')
#     print(g, X.shape)
#     document_term_matrix=pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())
#     yake_vocab[g]=vectorizer.vocabulary_
    
# # with open("yake_vocabulary.txt", "w") as text_file:
# #     text_file.write(str(yake_vocab))

## Representative Data Point
If you use k-means, there’s a function that returns the coordinates of the cluster center. You can then calculate the cosine distance of all the vectors in your cluster to this center point to find which data point is closest to the center. You can think of this as the best example or most representative data point in your cluster. Convert this data point back to its original form to get a better understanding of which types of requirements exist in this cluster.

In [ ]:
# #FIXME - This code is pulled from the referenced WSD project lines of code from the roadmap doc

# ### Look only at top 15 largest clusters ###

# df_clusters = pd.DataFrame(system_df.groupby(['labels']).count()['ID']).sort_values(
#     by='ID', ascending=False).reset_index()    
# df_clusters.rename(columns = {'ID': 'count'}, inplace=True)
# top15 = df_clusters['labels'][:15].tolist()
# system_df = system_df[system_df['labels'].isin(top15)].copy().reset_index(drop = True)
# df_clusters = df_clusters.merge(pd.DataFrame(system_df.groupby(['labels']).sum()['Elevated']),
#                                 on='labels', how='left')
# ### Find cluster centers ###

# indices = []
# example_tix_titles = []
# example_tix_summaries = []
# for label in top15:
#     c = model.cluster_centers_[label]
#     Y = np.array([np.array(x) for x in system_df['inference']])
#     cosine_distances = []
#     for row in Y:
#         cosine_distances.append(spatial.distance.cosine(row, c))
#     num_examples = min(int(df_clusters[df_clusters['labels']==label]['count'].values[0]), 3)
#     indices.append([cosine_distances.index(sorted(cosine_distances)[i]) for i in range(num_examples)]) ## three examples closest to cluster center
# for sublist in indices:
#     example_tix_titles.append([str(system_df.iloc[i]['ID']) + ': ' + str(system_df.iloc[i]['Title']) for i in sublist])
#     example_tix_summaries.append([utils.preprocess_text(str(system_df.iloc[i]['Brief_x0020_description']), keep_original=True, 
#                                                        display=True) for i in sublist])
# temp_dict = {'labels': top15, 'example_tix_ids': example_tix_titles, 'example_tix_summaries':example_tix_summaries}
# temp = pd.DataFrame(temp_dict).reset_index(drop=True)

# system_df = system_df.merge(temp, how = 'left', on = 'labels')
